In [3]:
import numpy as np
import pandas as pd

In [4]:
file_path = '../dataset/carbon_emission_dataset2.xlsx'
df=pd.read_excel(file_path)

In [5]:
print("Dataset Loaded Successfully")
print("Shape",df.shape)



Dataset Loaded Successfully
Shape (2109, 18)


In [6]:
df.sample(50)

,Mode,Logistics_partner,vehicle_type,fuel_type,distance_in_km per route,vehicle_capacity_in_kg,load_capacity_in_kg,average_speed_in_km_per_hr,traffic_condition,no_of_stop,vehicle_age_in_years,engine/norm_type,adjusted_fuel_efficiency_in_km/litre,adjusted_energy_kWh_per_km,total_energy_kWh,fuel_consumption_in_litre,load_factor,C02_emission_kg
814,Road,DTDC,Van,Diesel,80.22,1000.0,890.00,28.41,medium,27,4,BS-4,11.764,NaN,NaN,6.819,0.890000,15.6840
1884,Rail,Adani logistics,Cargo Train,Diesel,316.00,1632269.0,886570.00,39.00,medium,9,22,NaN,3.380,NaN,NaN,93.490,0.540000,250.5500
1212,Road,Flipkart,Light Truck,Diesel,115.40,2000.0,400.00,33.10,medium,7,5,BS-3,6.969,NaN,NaN,16.559,0.200000,39.7420
611,Road,Ecom Express,Van,Electric,128.74,700.0,674.35,27.69,light,76,0,Electric,NaN,0.03940,5.0724,NaN,0.963400,4.1594
2031,Air Cargo,SpiceXpress,Spice Jet Cargo,Jet Fuel,1404.00,21984.0,20456.00,884.00,high,1,1,Turboprop,3.280,NaN,NaN,427.640,0.930000,1351.3400
822,Road,DTDC,Van,Diesel,113.73,1200.0,612.00,36.76,light,15,2,BS-3,13.637,NaN,NaN,8.340,0.510000,20.0160
956,Road,Delhivery,Light Truck,Diesel,77.85,2200.0,1166.00,28.09,medium,20,6,BS-6,7.521,NaN,NaN,10.351,0.530000,21.7370
1252,Road,Flipkart,Light Truck,LNG,534.22,13000.0,7744.59,61.21,heavy,16,6,BS-IV,3.203,NaN,NaN,166.790,0.595700,458.6700
188,Road,XpressBees,Bike,Petrol,158.64,110.0,42.61,33.37,light,49,2,BS-6,43.310,NaN,NaN,3.660,0.387400,7.6900
2019,Air Cargo,SpiceXpress,Spice Jet Cargo,Jet Fuel,1170.00,90123.0,16978.00,672.00,moderate,2,16,Turboprop,3.410,NaN,NaN,342.650,0.890000,1082.7600


In [7]:
df.head(50)

,Mode,Logistics_partner,vehicle_type,fuel_type,distance_in_km per route,vehicle_capacity_in_kg,load_capacity_in_kg,average_speed_in_km_per_hr,traffic_condition,no_of_stop,vehicle_age_in_years,engine/norm_type,adjusted_fuel_efficiency_in_km/litre,adjusted_energy_kWh_per_km,total_energy_kWh,fuel_consumption_in_litre,load_factor,C02_emission_kg
0,Road,Delhivery,Bike,Petrol,91.00,118.0,61.31,30.0,medium,4,3,BS-VI,47.06,NaN,NaN,1.93,0.52,4.05
1,Road,Delhivery,Bike,Petrol,114.00,90.0,43.23,45.0,light,7,2,BS-IV,45.17,NaN,NaN,2.52,0.48,5.80
2,Road,Delhivery,Bike,Petrol,41.00,117.0,86.19,38.0,heavy,7,4,BS-III,43.44,NaN,NaN,0.94,0.74,2.26
3,Road,Delhivery,Bike,Petrol,98.00,128.0,90.79,45.0,medium,5,4,BS-VI,43.53,NaN,NaN,2.25,0.71,4.73
4,Road,Delhivery,Bike,Petrol,90.00,82.0,35.68,38.0,heavy,6,2,BS-IV,47.78,NaN,NaN,1.88,0.44,4.32
5,Road,Delhivery,Bike,Petrol,92.00,88.0,65.84,45.0,light,5,1,BS-IV,47.42,NaN,NaN,1.94,0.75,4.46
6,Road,Delhivery,Bike,Petrol,43.00,119.0,95.68,38.0,medium,7,2,BS-IV,44.28,NaN,NaN,0.97,0.80,2.23
7,Road,Delhivery,Bike,Petrol,73.00,123.0,108.90,45.0,light,6,1,BS-IV,42.03,NaN,NaN,1.74,0.89,4.00
8,Road,Delhivery,Bike,Petrol,119.00,119.0,54.97,38.0,medium,6,4,BS-III,46.72,NaN,NaN,2.55,0.46,6.12
9,Road,Delhivery,Bike,Petrol,68.00,120.0,38.68,45.0,heavy,3,4,BS-VI,47.11,NaN,NaN,1.44,0.32,3.02


In [8]:
df.tail(50)

,Mode,Logistics_partner,vehicle_type,fuel_type,distance_in_km per route,vehicle_capacity_in_kg,load_capacity_in_kg,average_speed_in_km_per_hr,traffic_condition,no_of_stop,vehicle_age_in_years,engine/norm_type,adjusted_fuel_efficiency_in_km/litre,adjusted_energy_kWh_per_km,total_energy_kWh,fuel_consumption_in_litre,load_factor,C02_emission_kg
2059,Air Cargo,SpiceXpress,Spice Jet Cargo,Jet Fuel,1825.0,14331.0,12995.0,780.0,low,3,8,Jet,5.88,NaN,NaN,310.51,0.91,981.20
2060,Air Cargo,SpiceXpress,Spice Jet Cargo,Jet Fuel,1308.0,10165.0,2552.0,674.0,moderate,4,20,Turboprop,3.41,NaN,NaN,383.15,0.25,1210.76
2061,Air Cargo,SpiceXpress,Spice Jet Cargo,Jet Fuel,638.0,31380.0,29399.0,823.0,low,1,10,Turboprop,3.24,NaN,NaN,196.99,0.94,622.49
2062,Air Cargo,SpiceXpress,Spice Jet Cargo,Jet Fuel,1378.0,15013.0,3094.0,710.0,low,4,15,Turbofan,4.87,NaN,NaN,283.14,0.21,894.73
2063,Air Cargo,SpiceXpress,Spice Jet Cargo,Jet Fuel,1909.0,19011.0,13086.0,824.0,moderate,3,21,Turboprop,3.47,NaN,NaN,550.11,0.69,1738.35
2064,Air Cargo,SpiceXpress,Spice Jet Cargo,Jet Fuel,834.0,12204.0,8919.0,680.0,low,1,24,Turboprop,3.43,NaN,NaN,243.20,0.73,768.52
2065,Air Cargo,SpiceXpress,Spice Jet Cargo,Jet Fuel,881.0,19180.0,10752.0,638.0,high,4,9,Turbofan,5.05,NaN,NaN,174.38,0.56,551.04
2066,Air Cargo,SpiceXpress,Spice Jet Cargo,Jet Fuel,1041.0,14907.0,12553.0,669.0,low,1,12,Turbofan,4.89,NaN,NaN,212.90,0.84,672.76
2067,Air Cargo,SpiceXpress,Spice Jet Cargo,Jet Fuel,1974.0,11885.0,6730.0,891.0,moderate,3,19,Jet,6.29,NaN,NaN,314.04,0.57,992.36
2068,Air Cargo,SpiceXpress,Spice Jet Cargo,Jet Fuel,512.0,12830.0,11154.0,622.0,moderate,3,5,Turbofan,4.87,NaN,NaN,105.11,0.87,332.15


In [9]:
df.describe()

,distance_in_km per route,vehicle_capacity_in_kg,load_capacity_in_kg,average_speed_in_km_per_hr,no_of_stop,vehicle_age_in_years,adjusted_fuel_efficiency_in_km/litre,adjusted_energy_kWh_per_km,total_energy_kWh,fuel_consumption_in_litre,load_factor,C02_emission_kg
count,2109.000000,2.109000e+03,2.109000e+03,2109.000000,2109.000000,2109.000000,1542.000000,567.000000,567.000000,1542.000000,2109.000000,2109.000000
mean,280.574329,9.146133e+04,5.893160e+04,87.923561,20.339023,5.989569,13.101877,0.072435,8.158010,66.974589,0.611091,138.685223
std,351.572256,3.531117e+05,2.324779e+05,172.731852,20.521343,5.632641,21.343438,0.418083,8.118228,103.848077,0.300444,280.747239
min,10.000000,1.300000e+01,1.340000e+01,18.080000,1.000000,0.000000,1.000000,0.031410,1.100000,0.810000,0.200000,1.017000
25%,96.930000,6.000000e+02,3.269900e+02,30.710000,7.000000,2.000000,3.984750,0.037120,3.207000,8.367750,0.451400,6.141600
50%,142.680000,1.800000e+03,9.000000e+02,37.430000,13.000000,5.000000,7.661500,0.039900,5.119900,17.783000,0.630900,25.087000
75%,277.800000,1.200000e+04,6.730000e+03,50.040000,25.000000,7.000000,13.208000,0.043325,8.423400,84.515000,0.801500,98.780000
max,2345.700000,1.999344e+06,1.750859e+06,894.000000,120.000000,80.000000,670.000000,9.000000,50.780000,789.000000,9.000000,1976.410000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109 entries, 0 to 2108
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Mode                                  2109 non-null   object 
 1   Logistics_partner                     2109 non-null   object 
 2   vehicle_type                          2109 non-null   object 
 3   fuel_type                             2109 non-null   object 
 4   distance_in_km per route              2109 non-null   float64
 5   vehicle_capacity_in_kg                2109 non-null   float64
 6   load_capacity_in_kg                   2109 non-null   float64
 7   average_speed_in_km_per_hr            2109 non-null   float64
 8   traffic_condition                     2109 non-null   object 
 9   no_of_stop                            2109 non-null   int64  
 10  vehicle_age_in_years                  2109 non-null   int64  
 11  engine/norm_type 

CLEANING COLUMN NAMES

In [11]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('/', '_')
)
print("Column names cleaned",df.columns)

Column names cleaned Index(['mode', 'logistics_partner', 'vehicle_type', 'fuel_type',
       'distance_in_km_per_route', 'vehicle_capacity_in_kg',
       'load_capacity_in_kg', 'average_speed_in_km_per_hr',
       'traffic_condition', 'no_of_stop', 'vehicle_age_in_years',
       'engine_norm_type', 'adjusted_fuel_efficiency_in_km_litre',
       'adjusted_energy_kwh_per_km', 'total_energy_kwh',
       'fuel_consumption_in_litre', 'load_factor', 'c02_emission_kg'],
      dtype='object')


CHECKING NULL VALUES

In [12]:
print(df[['fuel_type', 'adjusted_energy_kwh_per_km', 'total_energy_kwh']].isnull().sum())

# Preview a few electric vehicle rows
df[df['fuel_type'].str.lower().str.contains('electric', na=False)][['adjusted_energy_kwh_per_km', 'total_energy_kwh']].head(10)


fuel_type                        0
adjusted_energy_kwh_per_km    1542
total_energy_kwh              1542
dtype: int64


,adjusted_energy_kwh_per_km,total_energy_kwh
49,0.0409,4.38
50,0.0384,4.03
51,0.0362,1.81
52,0.0405,3.28
53,0.0395,3.44
54,0.0415,2.69
55,0.0402,2.77
56,0.0363,2.47
57,0.0395,2.37
58,0.0374,2.10


In [13]:
#MAKING A COPY
df_cleaned = df.copy()

In [14]:
print("\nEnergy column null counts in df_cleaned:")
print(df_cleaned[['adjusted_energy_kwh_per_km', 'total_energy_kwh']].isnull().sum())



Energy column null counts in df_cleaned:
adjusted_energy_kwh_per_km    1542
total_energy_kwh              1542
dtype: int64


In [15]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109 entries, 0 to 2108
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   mode                                  2109 non-null   object 
 1   logistics_partner                     2109 non-null   object 
 2   vehicle_type                          2109 non-null   object 
 3   fuel_type                             2109 non-null   object 
 4   distance_in_km_per_route              2109 non-null   float64
 5   vehicle_capacity_in_kg                2109 non-null   float64
 6   load_capacity_in_kg                   2109 non-null   float64
 7   average_speed_in_km_per_hr            2109 non-null   float64
 8   traffic_condition                     2109 non-null   object 
 9   no_of_stop                            2109 non-null   int64  
 10  vehicle_age_in_years                  2109 non-null   int64  
 11  engine_norm_type 

In [16]:
df_cleaned.replace(['N/A', 'n/a', 'NA', '-', '--'], np.nan, inplace=True)
print("Placeholder values replaced with NaN",df.head(100))

Placeholder values replaced with NaN     mode logistics_partner vehicle_type fuel_type  distance_in_km_per_route  \
0   Road         Delhivery         Bike    Petrol                      91.0   
1   Road         Delhivery         Bike    Petrol                     114.0   
2   Road         Delhivery         Bike    Petrol                      41.0   
3   Road         Delhivery         Bike    Petrol                      98.0   
4   Road         Delhivery         Bike    Petrol                      90.0   
..   ...               ...          ...       ...                       ...   
95  Road      Ecom Express         Bike    Petrol                     141.0   
96  Road      Ecom Express         Bike    Petrol                      94.0   
97  Road      Ecom Express         Bike    Petrol                     133.0   
98  Road      Ecom Express         Bike    Petrol                      97.0   
99  Road      Ecom Express         Bike    Petrol                      79.0   

    vehicle_ca

In [17]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109 entries, 0 to 2108
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   mode                                  2109 non-null   object 
 1   logistics_partner                     2109 non-null   object 
 2   vehicle_type                          2109 non-null   object 
 3   fuel_type                             2109 non-null   object 
 4   distance_in_km_per_route              2109 non-null   float64
 5   vehicle_capacity_in_kg                2109 non-null   float64
 6   load_capacity_in_kg                   2109 non-null   float64
 7   average_speed_in_km_per_hr            2109 non-null   float64
 8   traffic_condition                     2109 non-null   object 
 9   no_of_stop                            2109 non-null   int64  
 10  vehicle_age_in_years                  2109 non-null   int64  
 11  engine_norm_type 

CHECKING DUPLICATES

In [18]:
df_cleaned[df_cleaned.duplicated()]
df_cleaned.duplicated().sum()

np.int64(0)

In [19]:
before = df_cleaned.shape[0]
df_cleaned.drop_duplicates(inplace=True)
after = df_cleaned.shape[0]
print(f"Dropped {before - after} duplicate rows")

Dropped 0 duplicate rows


In [20]:
print("\n Missing values per column:\n")
print(df_cleaned.isnull().sum().sort_values(ascending=False))


 Missing values per column:

adjusted_energy_kwh_per_km              1542
total_energy_kwh                        1542
adjusted_fuel_efficiency_in_km_litre     567
fuel_consumption_in_litre                567
engine_norm_type                         114
mode                                       0
logistics_partner                          0
vehicle_type                               0
distance_in_km_per_route                   0
fuel_type                                  0
no_of_stop                                 0
traffic_condition                          0
average_speed_in_km_per_hr                 0
load_capacity_in_kg                        0
vehicle_capacity_in_kg                     0
vehicle_age_in_years                       0
load_factor                                0
c02_emission_kg                            0
dtype: int64


In [21]:
df_cleaned['logistics_partner'].value_counts()


logistics_partner
Delhivery              294
BlueDart               283
Ecom Express           195
Amazon                 170
Flipkart               164
XpressBees             150
Shadowfax              100
CONCOR                  89
Gati                    88
VRL Logistics           84
Adani logistics         84
TCL Freight             80
DTDC                    75
Gateway Rai Freight     73
LoadShare               70
SpiceXpress             51
Safeexpress             30
Rivigo                  29
Name: count, dtype: int64

COUNT VALUES

In [22]:
df_cleaned['logistics_partner'] = df_cleaned['logistics_partner'].str.strip().str.lower()
df_cleaned['logistics_partner'] = df_cleaned['logistics_partner'].replace({
    'bluedart': 'bluedart',
    'ecom express': 'ecom express',
    'xpressbees': 'xpressbees',
    'shadowfax': 'shadowfax',
    'vrl logistics': 'vrl logistics',
    'tcl freight': 'tcl freight',
    'dtdc': 'dtdc',
    'loadshare': 'loadshare',
    'adani logistics': 'adani logistics',
    'concor': 'concor',
    'spicexpress': 'spicexpress',
    'gateway rai freight': 'gateway rail freight',  #typo
    'safeexpress': 'safeexpress',
    'quickjet airlines': 'quickjet airlines',
    'gati': 'gati',
    'amazon': 'amazon',
    'flipkart': 'flipkart',
    'delhivery': 'delhivery',
    'rivigo': 'rivigo',
})
df_cleaned['logistics_partner'] = df_cleaned['logistics_partner'].str.title()
print(df_cleaned['logistics_partner'].value_counts())


logistics_partner
Delhivery               294
Bluedart                283
Ecom Express            195
Amazon                  170
Flipkart                164
Xpressbees              150
Shadowfax               100
Concor                   89
Gati                     88
Vrl Logistics            84
Adani Logistics          84
Tcl Freight              80
Dtdc                     75
Gateway Rail Freight     73
Loadshare                70
Spicexpress              51
Safeexpress              30
Rivigo                   29
Name: count, dtype: int64


In [23]:
df_cleaned['vehicle_type'].value_counts()


vehicle_type
Van                        450
Light Truck                374
Bike                       299
Medium Truck               266
Cargo Train                246
Auto                       180
Heavy Truck                135
Cargo Plane                 68
Spice Jet Cargo             51
Quickjet Amazon Partner     40
Name: count, dtype: int64

In [24]:
df_cleaned['vehicle_type'] = df_cleaned['vehicle_type'].str.strip().str.lower()
df_cleaned['vehicle_type'] = df_cleaned['vehicle_type'].replace({
    'van': 'van',
    'light truck': 'light_truck',
    'medium truck': 'medium_truck',
    'heavy truck': 'heavy_truck',
    'bike': 'bike',
    'auto': 'auto',
    'cargo train': 'train',
    'spice jet cargo': 'air_cargo',
    'cargo plane': 'air_cargo',
    'amazon air partner': 'air_cargo',
})
df_cleaned['vehicle_type'] = df_cleaned['vehicle_type'].astype('category')
print("\nStandardized Vehicle Types:\n", df_cleaned['vehicle_type'].value_counts())



Standardized Vehicle Types:
 vehicle_type
van                        450
light_truck                374
bike                       299
medium_truck               266
train                      246
auto                       180
heavy_truck                135
air_cargo                  119
quickjet amazon partner     40
Name: count, dtype: int64


In [25]:
df_cleaned['fuel_type'].unique()

array(['Petrol', 'Electric', 'LNG', 'CNG', 'Diesel', 'Hybrid',
       'Elecrtric', 'Jet Fuel'], dtype=object)

In [26]:
df_cleaned['fuel_type'] = df_cleaned['fuel_type'].astype(str).str.lower().str.strip()

df_cleaned['fuel_type'] = df_cleaned['fuel_type'].replace({
    'elecrtric': 'electric',
    'ev': 'electric',
})


In [27]:
print(df_cleaned['fuel_type'].value_counts())


fuel_type
diesel      878
electric    567
petrol      248
cng         162
jet fuel    159
lng          80
hybrid       15
Name: count, dtype: int64


CLEANING NULL VALUES

In [28]:
electric_mask = df_cleaned['fuel_type'].str.contains('electric', na=False)

# Nullify energy data for non-electric vehicles
df_cleaned.loc[~electric_mask, 'adjusted_energy_kwh_per_km'] = np.nan
df_cleaned.loc[~electric_mask, 'total_energy_kwh'] = np.nan

# Confirm
print(df_cleaned[['adjusted_energy_kwh_per_km', 'total_energy_kwh']].isnull().sum())


adjusted_energy_kwh_per_km    1542
total_energy_kwh              1542
dtype: int64


In [29]:
df_cleaned['traffic_condition'] = df_cleaned['traffic_condition'].str.lower().str.strip().str.replace('-', '', regex=False)
traffic_map = {
    'medium': 'moderate',
    'light': 'low',
    'high': 'heavy'
}

df_cleaned['traffic_condition'] = df_cleaned['traffic_condition'].replace(traffic_map)
df_cleaned['traffic_condition'] = df_cleaned['traffic_condition'].str.capitalize()



In [30]:
print(df_cleaned['traffic_condition'].value_counts())

traffic_condition
Moderate    940
Low         614
Heavy       555
Name: count, dtype: int64


In [31]:
df_cleaned['engine_norm_type'] = df_cleaned['engine_norm_type'].str.upper().str.strip().str.replace('-', '', regex=False)

# Step 2: Create mapping dictionary
norm_map = {
    'BS3': 'BS-III',
    'BSIII': 'BS-III',
    'BS4': 'BS-IV',
    'BSIV': 'BS-IV',
    'BS6': 'BS-VI',
    'BSVI': 'BS-VI',
    'ELECTRIC': 'Electric',
    'TURBOPROP': 'Turboprop',
    'TURBOFAN': 'Turbofan',
    'JET': 'Jet',
    'NA': 'N/A',  # If NA means Electric
    'N/A': 'N/A',
    'NONE': 'N/A',
    np.nan: 'N/A'
}

df_cleaned['engine_norm_type'] = df_cleaned['engine_norm_type'].replace(norm_map)




In [32]:
df_cleaned['engine_norm_type'] = df_cleaned['engine_norm_type'].fillna('Unknown')
df_cleaned['engine_norm_type'].sample(50)

705      Electric
769      Electric
30         BS-III
62       Electric
1116       BS-III
844         BS-VI
794        BS-III
1804          N/A
1058        BS-IV
1170        BS-VI
838         BS-IV
1591        BS-IV
1560       BS-III
1857     Electric
171      Electric
2087          Jet
1007        BS-VI
258         BS-IV
1948     Electric
31         BS-III
1412        BS-IV
630        BS-III
1599        BS-VI
923        BS-III
1682       BS-III
446         BS-IV
1254        BS-IV
975         BS-IV
1663        BS-IV
1680        BS-VI
1547        BS-IV
903      Electric
448         BS-VI
499        BS-III
1171        BS-IV
2091    Turboprop
1596       BS-III
368         BS-IV
1552        BS-VI
954        BS-III
1630        BS-VI
1955          Jet
724         BS-IV
127      Electric
908      Electric
156         BS-IV
1221       BS-III
365         BS-VI
438      Electric
1285     Electric
Name: engine_norm_type, dtype: object

In [33]:
print("\nNormalized Engine Norm Types:\n", df_cleaned['engine_norm_type'].value_counts())


Normalized Engine Norm Types:
 engine_norm_type
Electric     567
BS-VI        466
BS-IV        458
BS-III       345
N/A          114
Turboprop     63
Jet           54
Turbofan      42
Name: count, dtype: int64


In [34]:
df_cleaned.loc[~electric_mask, 'fuel_consumption_in_litre'] = (
    df_cleaned[~electric_mask]
    .groupby('vehicle_type')['fuel_consumption_in_litre']
    .transform(lambda x: x.fillna(x.mean()))
)


C:\Users\acer\AppData\Local\Temp\ipykernel_3924\894270800.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('vehicle_type')['fuel_consumption_in_litre']


In [35]:
df_cleaned['adjusted_fuel_efficiency_in_km_litre'].describe()


count    1542.000000
mean       13.101877
std        21.343438
min         1.000000
25%         3.984750
50%         7.661500
75%        13.208000
max       670.000000
Name: adjusted_fuel_efficiency_in_km_litre, dtype: float64

In [36]:
df_cleaned.loc[electric_mask, 'adjusted_fuel_efficiency_in_km_litre'] = np.nan


In [37]:
print(df_cleaned['adjusted_fuel_efficiency_in_km_litre'].isnull().sum())


567


MAPPING DATA TYPES OF FEATURES

In [38]:
df_cleaned['no_of_stop'] = df_cleaned['no_of_stop'].astype(int)
df_cleaned['vehicle_age_in_years'] = df_cleaned['vehicle_age_in_years'].astype(int)
df_cleaned['traffic_condition'] = df_cleaned['traffic_condition'].astype('category')
df_cleaned['mode'] = df_cleaned['mode'].astype('category')
df_cleaned['fuel_type'] = df_cleaned['fuel_type'].astype('category')
df_cleaned['vehicle_type'] = df_cleaned['vehicle_type'].astype('category')
df_cleaned['logistics_partner'] = df_cleaned['logistics_partner'].astype('category')
df_cleaned['distance_in_km_per_route'] = df_cleaned['distance_in_km_per_route'].astype(int)

In [39]:
df_cleaned[['distance_in_km_per_route', 'fuel_consumption_in_litre', 'adjusted_fuel_efficiency_in_km_litre', 'c02_emission_kg']].describe()


,distance_in_km_per_route,fuel_consumption_in_litre,adjusted_fuel_efficiency_in_km_litre,c02_emission_kg
count,2109.000000,1542.000000,1542.000000,2109.000000
mean,280.171645,66.974589,13.101877,138.685223
std,351.683515,103.848077,21.343438,280.747239
min,10.000000,0.810000,1.000000,1.017000
25%,96.000000,8.367750,3.984750,6.141600
50%,142.000000,17.783000,7.661500,25.087000
75%,277.000000,84.515000,13.208000,98.780000
max,2345.000000,789.000000,670.000000,1976.410000


In [40]:
print("\nFINAL MISSING VALUE REPORT:")
print(df_cleaned.isnull().sum().sort_values(ascending=False))



FINAL MISSING VALUE REPORT:
adjusted_energy_kwh_per_km              1542
total_energy_kwh                        1542
adjusted_fuel_efficiency_in_km_litre     567
fuel_consumption_in_litre                567
logistics_partner                          0
mode                                       0
vehicle_type                               0
fuel_type                                  0
distance_in_km_per_route                   0
vehicle_capacity_in_kg                     0
no_of_stop                                 0
traffic_condition                          0
average_speed_in_km_per_hr                 0
load_capacity_in_kg                        0
vehicle_age_in_years                       0
engine_norm_type                           0
load_factor                                0
c02_emission_kg                            0
dtype: int64


In [41]:
df_cleaned.to_csv("../dataset/carbon_emission_cleaned.csv", index=False)